### CodeExecutor로 로컬환경에서 코드 실행해보기
사용하는데 주의할점!
LocalCommandLineCodeExecutor는 실제로 내 로컬환경 코드가 실행되기 때문에
만약에 LLM이 위험한 코드를 작성하고 그것을 실행하게끔 만들었다면
내 PC가 위험한 상황을 맞이할 수 있으니 안전한 코드를 작성하는 에이전트 시스템의 경우에만 사용하는 걸 권장

좀 위험한 일이 있을것 같거나, 만약 외부라이브러리를 많이 설치해야되는 경우가 있다면
다커컨테이너??를 불러와서 거기서 실행시키면 된다.

In [ ]:
%pip install pyautogen

In [1]:
import warnings
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings('ignore')

In [4]:
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
# LocalCommandLineCodeExecutor: 내가 받은 PC 로컬환경 내에서 코드를 실행하고 그 결과물을 받아오겠다.


config_list = [{"model": "gpt-4o-mini"}]

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/codespace/.config/sagemaker/config.yaml


In [9]:
# UserProxyAgent와 AssistantAgent를 활용 할 것이다.
# UserProxyAgent에다가 CodeExecutor를 붙여서 UserProxyAgent가 어떤 코드실행 결과를 실행할 때 마다 사용자한테 허락을 받도록 구현 해보기

# 1. AssistantAgent 만들기 
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "config_list": config_list,  
        "temperature": 0, # 답변이 일관성 있게 나옴. (코드를 실행한다던가, 코드를 만드는데에 있어서 좀 더 높은 일관성을 가지고 싶을 때, 0으로 설정)
    }, 
)

# 2. UserProxyAgent 만들기 
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    max_consecutive_auto_reply=10, # 만약에 human_input_mode를 "NEVER"로 했을 때, 즉 인간의 개입이 없다고 했을 때 자동적으로 계속 UserProxyAgent가 응답을 하게되는데 그 최대치를 정해줌 (무한루프를 탈 수도 있기 때문에)
    # 이번엔 human_input_mode를 "ALWAYS"로 해서 항상 인간의 허락을 받고 진행할 거기 때문에 주석처리
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"), # UserProxyAgent의 메시지를 받았을 때, "content"에 해당하는 key 값을 value로 가져왔는데, "TERMINATE"로 끝나면 에이전트의 행동을 멈추겠다.
    code_execution_config={  # code_execution 사용
        # 생성된 코드를 실행하는 실행기 
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"), # work_dir(워크 디렉토리)를 설저앟고 "coding"이라는 폴더 안에 LocalCommandLineCodeExecutor가 실행한 코드의 결과물을 저장.
    },
    human_input_mode="NEVER", # NEVER: AI가 알아서 진행, ALWAYS: 인간에게 허락을 맡음
)

# 1) UserProxyAgent가 
chat_res = user_proxy.initiate_chat(
    assistant, # 3) assistant에이전트 한테
    message="""100까지 소수가 어떤 것이 있는지 코드로 살펴봐주세요.""", # 2) 해당 메세지를 보냄 누구한테?
    summary_method="reflection_with_llm", # user_proxy에이전트와 assistant에이전트가 대화한 내용을 요약해서 받기 위해 설정
)

user_proxy (to assistant):

100까지 소수가 어떤 것이 있는지 코드로 살펴봐주세요.

--------------------------------------------------------------------------------
assistant (to user_proxy):

소수를 찾기 위해 1부터 100까지의 숫자를 검사하는 파이썬 코드를 작성하겠습니다. 이 코드는 소수를 판별하고, 100 이하의 모든 소수를 출력할 것입니다.

아래 코드를 실행해 주세요.

```python
# filename: find_primes.py
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = [num for num in range(1, 101) if is_prime(num)]
print(primes)
```

코드를 실행한 후 결과를 확인해 주세요.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


--------------------------------------------------------------------------------
assistant (to user_proxy):

1

In [6]:
print("Chat history:", chat_res.chat_history) # 두 에이전트가 각각 어떤 content를 출력했는지에 대해서 쭉 담아놓음
print("-"*80)
print("Summary:", chat_res.summary) # 지금 까지 나눈 대화 요약
print("-"*80)
print("Cost info:", chat_res.cost) # 대화간에 발생했던 비용에 대해서 출력

Chat history: [{'content': '100까지 소수가 어떤 것이 있는지 코드로 살펴봐주세요.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': '소수를 찾기 위해 1부터 100까지의 숫자를 검사하는 파이썬 코드를 작성하겠습니다. 이 코드는 소수를 판별하고, 100 이하의 모든 소수를 출력할 것입니다.\n\n아래 코드를 실행해 주세요.\n\n```python\n# filename: find_primes.py\ndef is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True\n\nprimes = [num for num in range(1, 101) if is_prime(num)]\nprint(primes)\n```\n\n코드를 실행한 후 결과를 확인해 주세요.', 'role': 'user', 'name': 'assistant'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': '100 이하의 소수는 다음과 같습니다:\n\n\\[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97\\]\n\n작업이 완료되었습니다. TERMINATE', 'role': 'user', 'name': 'assistant'}]
------------

In [12]:
# followup of the previous question (이전 질문에 대한 후속 조치)

# 위에 initiate_chat을 통해서 대화를 시작했다. 대화에 이어서 메세지를 싶다면 아래와 같이 작성

user_proxy.send(
    recipient=assistant,
    message="""예시 영어 문장을 만들고, 키워드를 추출하세요""")

user_proxy (to assistant):

예시 영어 문장을 만들고, 키워드를 추출하세요

--------------------------------------------------------------------------------
assistant (to user_proxy):

예시 영어 문장을 만들고, 그 문장에서 키워드를 추출하는 파이썬 코드를 작성하겠습니다. 아래 코드를 실행해 주세요.

```python
# filename: extract_keywords_english.py
import re
from collections import Counter

# 예시 문장
sentence = "Machine learning is revolutionizing industries by providing insights from large datasets."

# 단어 추출 및 소문자 변환
words = re.findall(r'\b\w+\b', sentence.lower())

# 불용어 목록
stop_words = set([
    'is', 'the', 'by', 'to', 'from', 'and', 'a', 'of', 'in', 'that', 'it', 'as', 'for', 'on', 'with', 'this', 'an', 'are', 'at', 'be', 'or', 'which', 'but', 'not', 'all', 'can', 'we', 'you', 'your', 'their', 'they', 'if', 'has', 'have', 'more', 'than', 'so', 'about', 'up', 'out', 'just', 'like', 'will', 'no', 'do', 'what', 'when', 'where', 'who', 'how', 'there', 'here', 'my', 'our', 'its', 'he', 'she', 'them', 'me', 'us', 'him', 'her', 'his', 'hers', 'theirs', 'your